In [1]:
using Revise

In [2]:
using LazySets
using ModelVerification
using PyCall
using CSV
using ONNX
using Flux
using Test
using DataFrames

In [3]:
using Flux: onehotbatch, onecold, flatten
using Flux.Losses: logitcrossentropy
using Statistics: mean
using CUDA
using MLDatasets: CIFAR10
using MLUtils: splitobs, DataLoader
using Accessors

In [4]:
function test_mlp(prop_method)
    small_nnet_file = "../test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    # println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    # println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    # println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_violated    = Hyperrectangle(low = [19], high = [114]) # 20.0 ≤ y ≤ 90.0
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10.0 or 19 ≤ y
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 10.0 or 18 ≤ y
    info = nothing
    search_method = BFS(max_iter=1000, batch_size=13)
    split_method = Bisect(1)
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_violated)).status == :violated
end
test_mlp(Crown())

Test Failed at 

/home/tianhao/dev/ModelVerification.jl/tmp/test.ipynb:17
  Expression: (verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds))).status == :holds
   Evaluated: unknown == holds


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [13]:
# function test_conv(prop_method)
model = Chain([
    Conv((3, 3), 3 => 64, relu, pad=(1, 1), stride=(1, 1)),
    BatchNorm(64),
    MeanPool((2,2)),
    Conv((5, 5), 64 => 128, relu, pad=(2, 2), stride=(1, 1)),
    BatchNorm(128),
    MeanPool((2,2)),
    flatten,
    Dense(8192, 1024, relu),
    Dense(1024, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:5]
println(typeof(image_seeds[1][1,1,1,1]))
# out1 = model.layers[1](image_seeds)
# layer = model.layers[2]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
prop_method = ImageStarZono()
output_set = BallInf(zeros(10), 1.0)
verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))
# test_mlp(ImageStarZono())

┌ Error: Failed to revise /home/tianhao/dev/ModelVerification.jl/src/propagate/operators/relu.jl
│   exception = (ErrorException("lowering returned an error, \$(Expr(:error, \"{ } vector syntax is discontinued around /home/tianhao/dev/ModelVerification.jl/src/propagate/operators/relu.jl:65\"))"), Union{Ptr{Nothing}, Base.InterpreterIP}[])
└ @ Revise /home/tianhao/.julia/packages/Revise/P8ITN/src/packagedef.jl:717
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /home/tianhao/dev/ModelVerification.jl/src/propagate/operators/relu.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /home/tianhao/.julia/packages/Revise/P8ITN/src/packagedef.jl:825


Float32


before size: (32, 32, 64, 4)
after size: (32, 32, 64, 35249)


before size: (16, 16, 128, 4)
after size: (16, 16, 128, 17403)


MethodError: MethodError: no method matching forward_act(::ImageStarZono, ::typeof(relu), ::Vector{Zonotope{Float32, Vector{Float32}, Matrix{Float32}}}, ::Vector{Any})
Closest candidates are:
  forward_act(!Matched::Ai2h, ::typeof(relu), ::Vector{<:AbstractPolytope}, ::Any) at ~/dev/ModelVerification.jl/src/propagate/operators/relu.jl:60
  forward_act(!Matched::Box, ::typeof(relu), ::Vector{<:AbstractPolytope}, ::Any) at ~/dev/ModelVerification.jl/src/propagate/operators/relu.jl:70
  forward_act(::Any, ::typeof(relu), !Matched::ModelVerification.ImageZonoBound, ::Any) at ~/dev/ModelVerification.jl/src/propagate/operators/relu.jl:1
  ...